In [1]:
import os
import pandas as pd

In [41]:
from sklearn import tree, externals,linear_model

In [16]:
%load_ext gvmagic

The gvmagic extension is already loaded. To reload it, use:
  %reload_ext gvmagic


In [4]:
file_path = 'pets.csv'

In [5]:
df= pd.read_csv(os.path.join('.',file_path))

In [6]:
df

,name,animal,size,friendly
0,Bo,snake,small,True
1,Miles,dog,small,True
2,Fifi,cat,small,False
3,Muffy,cat,small,True
4,Rufus,dog,large,True
5,Jebediah,snail,small,True
6,Aloysius,dog,large,False
7,Tom,cat,large,False


In [7]:
X = df[['name','animal','size']]

In [8]:
c = df['friendly']

In [9]:
X

,name,animal,size
0,Bo,snake,small
1,Miles,dog,small
2,Fifi,cat,small
3,Muffy,cat,small
4,Rufus,dog,large
5,Jebediah,snail,small
6,Aloysius,dog,large
7,Tom,cat,large


In [10]:
c

0     True
1     True
2    False
3     True
4     True
5     True
6    False
7    False
Name: friendly, dtype: bool

In [11]:
names_df=pd.get_dummies(df.name)
animal_df = pd.get_dummies(df.animal)
size_df = pd.get_dummies(df['size'])

In [12]:
X = pd.concat([names_df, animal_df, size_df], axis = 1)

In [17]:
X

,Aloysius,Bo,Fifi,Jebediah,Miles,Muffy,Rufus,Tom,cat,dog,snail,snake,large,small
0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,1,0,0,0,0,1,0,0,0,1
2,0,0,1,0,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,1,0,0,1,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,1,0,0,1,0
5,0,0,0,1,0,0,0,0,0,0,1,0,0,1
6,1,0,0,0,0,0,0,0,0,1,0,0,1,0
7,0,0,0,0,0,0,0,1,1,0,0,0,1,0


In [13]:
model = tree.DecisionTreeClassifier().fit(X, c)

In [14]:
model.score(X,c)

1.0

In [15]:
dot_data = externals.six.StringIO()
tree.export_graphviz(model, out_file = dot_data, feature_names = X.columns)
%dotstr dot_data.getvalue()

WindowsError: [Error 2] The system cannot find the file specified

In [18]:
X = pd.concat([animal_df, size_df], axis = 1)

In [19]:
model = tree.DecisionTreeClassifier().fit(X, c)

In [20]:
dot_data = externals.six.StringIO()
tree.export_graphviz(model, out_file = dot_data, feature_names = X.columns)
%dotstr dot_data.getvalue()

WindowsError: [Error 2] The system cannot find the file specified

In [26]:
df['start_with_a']=df.name.str.startswith('A')

In [30]:
df['has_4_letter']= df.name.str.len()<=4

In [32]:
start_with_a_df =df.start_with_a

In [33]:
has_4_letter=df.has_4_letter

In [38]:
X = pd.concat([start_with_a_df,has_4_letter, animal_df, size_df], axis = 1)

In [40]:
X.drop(['snail','large'], axis=1, inplace=True)

In [42]:
model = linear_model.LogisticRegression().fit(X,c)

In [44]:
model.score(X,c)

0.875

In [45]:
model.coef_

array([[-0.48605308, -0.51800442, -0.46019235,  0.1516086 ,  0.31470857,
         0.70267803]])

In [46]:
import numpy as np


In [47]:
np.logspace(2,-4,10)

array([  1.00000000e+02,   2.15443469e+01,   4.64158883e+00,
         1.00000000e+00,   2.15443469e-01,   4.64158883e-02,
         1.00000000e-02,   2.15443469e-03,   4.64158883e-04,
         1.00000000e-04])

In [48]:
coefs_df = pd.DataFrame(columns=X.columns)

In [49]:
coefs_df

,start_with_a,has_4_letter,cat,dog,snake,small


In [51]:
for C in np.logspace(2,-4,10):
    model = linear_model.LogisticRegression(penalty='l1', C=C).fit(X,c)
    coefs= pd.Series(model.coef_[0], name = C, index = X.columns)
    coefs_df= coefs_df.append(coefs)

In [52]:
coefs_df

,start_with_a,has_4_letter,cat,dog,snake,small
100.000000,-8.862532,-8.693371,-1.172777,0.0,7.522166,1.497533
21.544347,-5.707640,-5.527497,-1.195035,0.0,4.332570,1.532400
4.641589,-2.149956,-1.918796,-1.348385,0.0,0.570042,1.783638
1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.405462
0.215443,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
0.046416,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
0.010000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
0.002154,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
0.000464,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
0.000100,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [55]:
coefs_df = pd.DataFrame(columns=X.columns)
for C in np.logspace(2,-4,10):
    model = linear_model.LogisticRegression(penalty='l2', C=C).fit(X,c)
    coefs= pd.Series(model.coef_[0], name = C, index = X.columns)
    coefs_df= coefs_df.append(coefs)

In [56]:
coefs_df

,start_with_a,has_4_letter,cat,dog,snake,small
100.000000,-5.620293,-5.142843,-2.321358,0.509330,3.448856,2.735899
21.544347,-3.389166,-3.070004,-1.707738,0.383348,1.961760,2.081485
4.641589,-1.554725,-1.462548,-1.062019,0.250711,0.886030,1.402292
1.000000,-0.486053,-0.518004,-0.460192,0.151609,0.314709,0.702678
0.215443,-0.112219,-0.125612,-0.121109,0.069466,0.089932,0.242623
0.046416,-0.023636,-0.024906,-0.024654,0.020628,0.022032,0.064234
0.010000,-0.005024,-0.005095,-0.005082,0.004867,0.004939,0.014722
0.002154,-0.001078,-0.001082,-0.001081,0.001071,0.001074,0.003218
0.000464,-0.000232,-0.000232,-0.000232,0.000232,0.000232,0.000696
0.000100,-0.000050,-0.000050,-0.000050,0.000050,0.000050,0.000150
